In [2]:
import numpy as np
import os

In [3]:
os.chdir("C:\\Users\\Bridget Leonard\\Desktop\\DNN_fMRI")

## Part 5: fMRI Prediction
- project features into opposite modal space with feature alignment
- predict fMRI activity using features

### 1 Feature alignment matrices

#### 1.1 Load the matrices
These matrices were calculated in part 3. They represent the linear relationship between image and caption pairs. The image-->caption matrices were estimated by predicting each language feature from visual features, the caption--> image matrices were estimated by predicting each visual feature from the language features. Thus we have two matrices with size [768, 768] where each column represents a language or visual feature's linear relationship (beta coefficient) with each visual or language feature, respectively.

In [60]:
caption_to_image_matrices = np.load("bridgetower\\data\\feature_alignment\\caption_to_image_matrices.npy")
image_to_caption_matrices = np.load("bridgetower\\data\\feature_alignment\\image_to_caption_matrices.npy")

In [61]:
caption_to_image_matrices.shape

(768, 768)

#### 1.2 Project data into the opposite modal space

In [62]:
# Load feature vectors
# movie data
test = np.load("bridgetower\\feature_vectors\\movie\\test_data.npy")
train00 = np.load("bridgetower\\feature_vectors\\movie\\train_00_data.npy")
train01 = np.load("bridgetower\\feature_vectors\\movie\\train_01_data.npy")
train02 = np.load("bridgetower\\feature_vectors\\movie\\train_02_data.npy")
train03 = np.load("bridgetower\\feature_vectors\\movie\\train_03_data.npy")
train04 = np.load("bridgetower\\feature_vectors\\movie\\train_04_data.npy")
train05 = np.load("bridgetower\\feature_vectors\\movie\\train_05_data.npy")
train06 = np.load("bridgetower\\feature_vectors\\movie\\train_06_data.npy")
train07 = np.load("bridgetower\\feature_vectors\\movie\\train_07_data.npy")
train08 = np.load("bridgetower\\feature_vectors\\movie\\train_08_data.npy")
train09 = np.load("bridgetower\\feature_vectors\\movie\\train_09_data.npy")
train10 = np.load("bridgetower\\feature_vectors\\movie\\train_10_data.npy")
train11 = np.load("bridgetower\\feature_vectors\\movie\\train_11_data.npy")

# story data
alternateithicatom = np.load("bridgetower\\feature_vectors\\story\\alternateithicatom_data.npy")
avatar = np.load("bridgetower\\feature_vectors\\story\\avatar_data.npy")
howtodraw = np.load("bridgetower\\feature_vectors\\story\\howtodraw_data.npy")
legacy = np.load("bridgetower\\feature_vectors\\story\\legacy_data.npy")
life = np.load("bridgetower\\feature_vectors\\story\\life_data.npy")
myfirstdaywiththeyankees = np.load("bridgetower\\feature_vectors\\story\\myfirstdaywiththeyankees_data.npy")
naked = np.load("bridgetower\\feature_vectors\\story\\naked_data.npy")
odetostepfather = np.load("bridgetower\\feature_vectors\\story\\odetostepfather_data.npy")
souls = np.load("bridgetower\\feature_vectors\\story\\souls_data.npy")
undertheinfluence = np.load("bridgetower\\feature_vectors\\story\\undertheinfluence_data.npy")

Visual to text: Movie data

In [64]:
test_transformed = np.dot(test, image_to_caption_matrices.T)
train00_transformed = np.dot(train00, image_to_caption_matrices.T)
train01_transformed = np.dot(train01, image_to_caption_matrices.T)
train02_transformed = np.dot(train02, image_to_caption_matrices.T)
train03_transformed = np.dot(train03, image_to_caption_matrices.T)
train04_transformed = np.dot(train04, image_to_caption_matrices.T)
train05_transformed = np.dot(train05, image_to_caption_matrices.T)
train06_transformed = np.dot(train06, image_to_caption_matrices.T)
train07_transformed = np.dot(train07, image_to_caption_matrices.T)
train08_transformed = np.dot(train08, image_to_caption_matrices.T)
train09_transformed = np.dot(train09, image_to_caption_matrices.T)
train10_transformed = np.dot(train10, image_to_caption_matrices.T)
train11_transformed = np.dot(train11, image_to_caption_matrices.T)

Text to visual: Story data

In [65]:
alternateithicatom_transformed = np.dot(alternateithicatom, caption_to_image_matrices.T)
avatar_transformed = np.dot(avatar, caption_to_image_matrices.T)
howtodraw_transformed = np.dot(howtodraw, caption_to_image_matrices.T)
legacy_transformed = np.dot(legacy, caption_to_image_matrices.T)
life_transformed = np.dot(life, caption_to_image_matrices.T)
myfirstdaywiththeyankees_transformed = np.dot(myfirstdaywiththeyankees, caption_to_image_matrices.T)
naked_transformed = np.dot(naked, caption_to_image_matrices.T)
odetostepfather_transformed = np.dot(odetostepfather, caption_to_image_matrices.T)
souls_transformed = np.dot(souls, caption_to_image_matrices.T)
undertheinfluence_transformed = np.dot(undertheinfluence, caption_to_image_matrices.T)

### 2 Load voxelwise encoding models
These matrices were calculated in part 4.

In [6]:
# Load alphas and coefficients from encoding models
best_alphas = np.load("results\\encoding_models\\movie\\best_alphas_1623-3244.npy")
coefficients = np.load("results\\encoding_models\\movie\\coefficients_1623-3244.npy")

print("Number of voxels processed:", np.count_nonzero(best_alphas))
print("Percentage of voxels processed: " + str(round((np.count_nonzero(best_alphas)/len(best_alphas))*100, 2)) + "%")

Number of voxels processed: 1622
Percentage of voxels processed: 100.0%


In [5]:
coefficients.shape

(1623, 3072)

For each voxel, we have two values:

1) The best alpha value to capture the linear relationship between this voxel and each of the 4k stimulus features (using L2-regularized linear regression)

2) A list of coefficients with length 4k (4*768 = 3072), giving us one slope for each of our features to show it's relation to the voxel's activity

In [7]:
print("Best alpha for our first voxel:", best_alphas[0])
print("Coefficients for the first 10 features for our first voxel:", coefficients[0][:10])
print("Number of coefficients for our first voxel:", len(coefficients[0]))

Best alpha for our first voxel: 33932.217718953296
Coefficients for the first 10 features for our first voxel: [-1.6447419e-04 -4.3410615e-05  1.0913535e-04  1.5899843e-04
 -5.2098444e-06  2.5727547e-04  1.7378540e-04 -3.2150943e-04
  2.9492809e-04  1.4711423e-04]
Number of coefficients for our first voxel: 3072


In [8]:
best_alphas[:27]

array([  33932.21771895, 1000000.        ,   19306.97728883,
        184206.99693267, 1000000.        ,   59636.23316595,
       1000000.        , 1000000.        , 1000000.        ,
       1000000.        , 1000000.        , 1000000.        ,
        104811.31341547, 1000000.        ,  568986.60290183,
       1000000.        , 1000000.        ,   19306.97728883,
       1000000.        ,   10985.41141988,  104811.31341547,
       1000000.        , 1000000.        , 1000000.        ,
       1000000.        , 1000000.        , 1000000.        ])